# CS 533 Project - TVCA analysis
by Abhishai Joy and Ravishankar

## Purpose and Context
The purpose of this assignment is to perform an exploratory analysis on [TVCA](http://www.biketreasurevalley.org/) data

- Obtain TVCA data from Mike Stefancic
- Set up a Jupyter notebook and data set to begin a new analysis
- Carry out an exploratory analysis to understand a data set’s contents and communicate them to others.


## Setup

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Loading Data

All binary files are in the `bin/` folder

In [5]:
bike_count = pd.ExcelFile('bin/bike-count-master-summary-updated-2019.xlsx')

The excel file has 22 different sheets, so we save each sheet as a pandas data frame

In [6]:
data_overall = pd.read_excel(bike_count,'Overall')
data_overall.head()

,Location,2007,2008,2009,2010,2011,2012-May,2012-September,2013-May,2013-September,2014-May,2014-September,2015-05-01 00:00:00,2019-05-01 00:00:00,Unnamed: 14,Total,Average
0,03rd / Bannock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,NaN,NaN,NaN,NaN,NaN,52,52
1,03rd / Myrtle,NaN,NaN,NaN,NaN,151.0,NaN,NaN,59.0,150.0,95.0,133.0,NaN,NaN,NaN,588,117.6
2,05th / Idaho,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,28
3,06th / O'Farrell,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,62
4,06th / Union,NaN,NaN,NaN,NaN,NaN,NaN,137.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,137


In [5]:
data_2007 = pd.read_excel(bike_count,'2007')
data_2007.head()

,Location,Date,Time,Total,Men,On Street,On Sidewalk,With Helmets,Without Helmets,Women,On Street.1,On Sidewalk.1,With Helmets.1,Without Helmets.1,Volunteer Name
0,8th / Fort,2007-05-08,AM,82.0,48.0,31.0,17.0,28.0,23.0,34.0,11.0,22.0,16.0,18.0,K Lundstrom
1,10th / Bannock,2007-05-10,AM,69.0,49.0,43.0,6.0,30.0,19.0,20.0,16.0,4.0,7.0,13.0,K Lundstrom
2,13th / Fort,2007-04-24,AM,51.0,35.0,23.0,12.0,27.0,8.0,16.0,7.0,9.0,10.0,6.0,M Andrew Tripp
3,23rd / State,2007-05-01,AM,18.0,6.0,2.0,4.0,3.0,3.0,12.0,0.0,12.0,8.0,4.0,M Andrew Tripp
4,27th / State,2007-05-09,PM,34.0,26.0,8.0,18.0,9.0,17.0,8.0,1.0,7.0,4.0,4.0,K Lundstrom


In [6]:
data_2008 = pd.read_excel(bike_count,'2008')
data_2008.head()

,Location,Date,Time,Total,Men,On Street,On Sidewalk,With Helmets,Without Helmets,Women,On Street.1,On Sidewalk.1,With Helmets.1,Without Helmets.1,Volunteer Name
0,8th / Fort,2008-09-23,PM,150.0,101.0,55.0,46.0,36.0,65.0,49.0,26.0,23.0,23.0,26.0,Steve Dunlop
1,10th / Bannock,2008-09-24,PM,174.0,116.0,107.0,9.0,39.0,77.0,58.0,45.0,13.0,22.0,36.0,Krista Broderick
2,13th / Fort,2008-09-23,PM,135.0,87.0,58.0,29.0,47.0,40.0,48.0,24.0,24.0,21.0,27.0,Walt Sledzieski
3,36th / Hill / Catalpa,2008-09-23,AM,46.0,32.0,20.0,12.0,21.0,11.0,14.0,8.0,6.0,10.0,4.0,Al Kristal
4,36th / Hill / Catalpa,2008-09-24,PM,50.0,46.0,39.0,7.0,37.0,9.0,4.0,3.0,1.0,3.0,1.0,Amar


In [7]:
data_2010 = pd.read_excel(bike_count,'2010')
data_2010.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,08th / Bannock,2010-09-16,AM,131.0,55.0,76.0,33.0,15.0,22.0,61.0,David Peckham
1,08th / Bannock,2010-09-14,PM,267.0,133.0,134.0,25.0,51.0,108.0,83.0,Tacy Hitchcock
2,08th / Fort,2010-09-15,PM,132.0,72.0,60.0,22.0,35.0,50.0,25.0,Cindy Ritchie
3,08th / Greenbelt,2010-09-14,PM,270.0,64.0,206.0,64.0,127.0,0.0,79.0,Mark Ritchie
4,08th / River,2010-09-15,AM,114.0,93.0,21.0,62.0,7.0,31.0,14.0,David Peckham


In [8]:
data_2011 = pd.read_excel(bike_count,'2011')
data_2011.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,03rd / Myrtle,2011-09-14,PM,151.0,110.0,41.0,52.0,23.0,58.0,18.0,Justin Lucas
1,08th / Bannock,2011-09-15,AM,115.0,44.0,71.0,26.0,28.0,18.0,43.0,Dave Fostch
2,08th / Greenbelt S,2011-09-13,PM,346.0,61.0,285.0,61.0,182.0,0.0,103.0,Amy Stahl
3,08th / River,2011-09-13,PM,249.0,203.0,46.0,104.0,35.0,99.0,11.0,Mark Ritchie
4,28th / Sunset,2011-09-14,AM,23.0,9.0,14.0,4.0,7.0,5.0,7.0,Sarah Lunstrum


In [9]:
data_2012_may = pd.read_excel(bike_count,'2012-May')
data_2012_may.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,08th / Bannock,2012-05-16,AM,148.0,56.0,92.0,19.0,29.0,37.0,63.0,Jim Pace
1,08th / Franklin,2012-05-17,AM,61.0,28.0,33.0,19.0,3.0,9.0,30.0,Andy Query
2,08th / River,2012-05-16,AM,115.0,93.0,22.0,42.0,13.0,51.0,9.0,Dan Conlin
3,10th /Bannock,2012-05-16,PM,185.0,60.0,125.0,27.0,80.0,33.0,45.0,Naala Figuroas
4,10th / Franklin,2012-05-15,AM,113.0,32.0,81.0,28.0,10.0,4.0,71.0,Andy Query


In [10]:
data_2012_sept = pd.read_excel(bike_count,'2012-September')
data_2012_sept.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,05th / Idaho,2012-09-13,AM,28.0,13.0,15.0,4.0,12.0,9.0,3.0,Jim Pace
1,06th / O'Farrell,2012-09-13,PM,62.0,45.0,17.0,11.0,8.0,34.0,9.0,Vicki Trees
2,06th / Union,2012-09-11,PM,127.0,83.0,44.0,31.0,14.0,52.0,30.0,Naala Figueras
3,06th / Union,2012-09-12,AM,46.0,23.0,23.0,18.0,9.0,5.0,14.0,Naala Figueras
4,06th / Union,2012-09-15,PM,137.0,110.0,27.0,25.0,7.0,85.0,20.0,Vicki Trees


In [11]:
data_2013_may = pd.read_excel(bike_count,'2013-May')
data_2013_may.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,03rd / Myrtle,2013-05-07,AM,59.0,51.0,8.0,17.0,2.0,34.0,6.0,Tony Grange
1,08th / Bannock,2013-05-09,AM,149.0,50.0,99.0,15.0,25.0,35.0,74.0,Jim Pace
2,08th / Fort,2013-05-09,PM,143.0,89.0,54.0,16.0,35.0,73.0,19.0,Robert Moore
3,08th / N Greenbelt,2013-05-07,AM,200.0,74.0,126.0,56.0,44.0,18.0,82.0,Matt Bergstrom
4,08th / River,2013-05-08,AM,123.0,114.0,9.0,54.0,7.0,60.0,2.0,Mark Ritchie


In [12]:
data_2013_sept = pd.read_excel(bike_count,'2013-September')
data_2013_sept.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,03rd / Bannock,2013-09-10,AM,52.0,21.0,31.0,5.0,13.0,16.0,18.0,Andy Query
1,03rd / Myrtle,2013-09-10,PM,150.0,120.0,30.0,58.0,16.0,62.0,14.0,Kathleen Coskey
2,07th / Brumback,2013-09-11,AM,24.0,4.0,20.0,2.0,16.0,2.0,4.0,Kay Hummel
3,08th / Bannock,2013-09-10,AM,165.0,62.0,103.0,27.0,31.0,35.0,72.0,Jim Pace
4,08th / Bannock,2013-09-11,AM,149.0,68.0,81.0,28.0,24.0,40.0,57.0,Jim Pace


In [13]:
data_2014_may = pd.read_excel(bike_count,'2014-May')
data_2014_may.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,03rd / Myrtle,2014-05-08,AM,95.0,84.0,11.0,26.0,3.0,58.0,8.0,Marcus Orton
1,08th / Bannock,2014-05-07,AM,111.0,36.0,75.0,14.0,28.0,22.0,47.0,Jim Pace
2,08th / Bannock,2014-05-07,PM,234.0,116.0,118.0,35.0,62.0,81.0,56.0,Vicki Trees
3,08th / Fort,2014-05-07,PM,109.0,67.0,42.0,6.0,28.0,61.0,14.0,Karry Fischer
4,08th / Main,2014-05-08,PM,276.0,175.0,101.0,75.0,20.0,100.0,81.0,Gracie Bingham


In [14]:
data_2014_sept = pd.read_excel(bike_count,'2014-September')
data_2014_sept.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,03rd / Myrtle,2014-09-10,PM,133.0,105.0,28.0,46.0,14.0,59.0,14.0,Jean Cardeno
1,03rd / Myrtle,2014-09-11,AM,87.0,72.0,15.0,33.0,6.0,39.0,9.0,Kelly Paananen
2,08th / Bannock,2014-09-10,AM,201.0,41.0,160.0,11.0,36.0,30.0,124.0,Jim Pace
3,08th / Bannock,2014-09-11,PM,306.0,145.0,161.0,35.0,61.0,110.0,100.0,Jim Pace
4,08th / Brumback,2014-09-11,AM,30.0,19.0,11.0,12.0,10.0,7.0,1.0,Kristin Little


In [16]:
data_2015_may_jub = pd.read_excel(bike_count,'2015-May JUB')
data_2015_may_jub.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,36th / Taft,2015-05-27,AM,22.0,13.0,9.0,10.0,5.0,3.0,4.0,Michael Van Lydegraf
1,Bogart / Caswell,2015-05-27,PM,8.0,6.0,2.0,3.0,0.0,3.0,2.0,Kristi Watkins
2,Bogart / State,2015-05-27,AM,6.0,3.0,3.0,2.0,1.0,1.0,2.0,Kristi Watkins
3,Collister / State,2015-05-28,AM,17.0,8.0,9.0,8.0,3.0,0.0,6.0,Michael Van Lydegraf
4,Gary / Gillis,2015-05-27,PM,29.0,19.0,10.0,10.0,3.0,9.0,7.0,Travis Jeffers


In [17]:
data_2016_may = pd.read_excel(bike_count,'2016- May')
data_2016_may.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name,Unnamed: 11
0,Bown/Parkcenter,2016-05-11,AM,36,11,25.0,0,22.0,11.0,3.0,Steve Dunlap,NaN
1,Beacon/Boise,2016-05-11,AM,94,46,48.0,11,34.0,35.0,14.0,Bryan Wilkins,NaN
2,27th/State,2016-05-12,PM,53,23,30.0,6,13.0,17.0,17.0,Jean Coadeño,NaN
3,Veterans Memorial/Chinden,2016-05-11,PM,62,36,26.0,15,16.0,21.0,10.0,Robert McGarvey,NaN
4,Hill/Horseshoe Bend,2016-05-11,AM,18,7,11.0,7,7.0,0.0,4.0,Ailen Fry,NaN


In [18]:
data_2018_may = pd.read_excel(bike_count,'2018 - May')
data_2018_may.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,Federal/Protest/Kootenia,2018-05-09,am,66.0,24.0,42.0,5.0,17.0,19.0,25.0,Janell Ruthe
1,10th and Franklin/Boise High,2018-05-09,am,148.0,54.0,94.0,43.0,10.0,11.0,84.0,Joanie Fauci
2,Garden St/Franklin Rd,2019-05-10,PM,33.0,20.0,13.0,10.0,4.0,10.0,9.0,Christa Howarth
3,27th/State St,2018-05-09,PM,45.0,31.0,14.0,11.0,9.0,20.0,5.0,Lisa Ozuna
4,10th/Bannock,2018-05-01,PM,206.0,74.0,132.0,26.0,69.0,48.0,63.0,Kelly Frazier


In [19]:
data_2019_may = pd.read_excel(bike_count,'2019 - May')
data_2019_may.head()

,Location,Date,Time,Total,Total: Location 1,Total: Location 2,Leaving Leg A,Leaving Leg B,Leaving Leg C,Leaving Leg D,Volunteer Name
0,8th/Bannock,2019-05-07,AM,130.0,32,98.0,13.0,27.0,19.0,71.0,Brian Wilkens
1,8th/fort,2019-05-07,AM,43.0,26,17.0,14.0,10.0,12.0,7.0,April Fredrickson
2,8th/river,2019-05-07,PM,282.0,239,43.0,169.0,30.0,70.0,13.0,Clancy Anderson
3,Anne frank/greenbelt,2019-05-08,AM,156.0,4,152.0,4.0,63.0,0.0,89.0,Kristen Little
4,Latah/Kootani,2019-05-07,PM,34.0,23,11.0,11.0,6.0,12.0,5.0,Greg Little


In [20]:
'''
Arguments
---------
dbfile  : DBF file - Input to be imported
upper   : Condition - If true, make column heads upper case
'''
def dbf2DF(dbfile, upper=True): #Reads in DBF files and returns Pandas DF
    db = ps.open(dbfile) #Pysal to open DBF
    d = {col: db.by_col(col) for col in db.header} #Convert dbf to dictionary
    #pandasDF = pd.DataFrame(db[:]) #Convert to Pandas DF
    pandasDF = pd.DataFrame(d) #Convert to Pandas DF
    if upper == True: #Make columns uppercase if wanted 
        pandasDF.columns = map(str.upper, db.header) 
    db.close() 
    return pandasDF

In [21]:
df = dbf2DF('Bike\ Data\ \(CS\ 533\)\ACHD\ Traffic\ Counts.dbf')

AttributeError: module 'pysal' has no attribute 'open'